### link to source https://medium.com/@anand0427/network-graph-with-at-t-data-using-plotly-a319f9898a02

In [119]:
import csv
import pandas as pd
import networkx as nx
import plotly.graph_objs as go
#from plotly.plotly import iplot

In [120]:
df=pd.read_csv('SharedDrive.csv')
df.head()

,FIMGroup,FileName
0,FORDNA1\LROSENB9,\\ECC9000106.md2.ford.com\qlik_prod8422\Descri...
1,FORDNA1\BALASRI,\\ECC9000106.md2.ford.com\qlik_prod8422\Descri...
2,FORDNA2\JDAVI273,\\ECC9000106.md2.ford.com\qlik_prod8422\Descri...
3,FORDNA1\LROSENB9,\\ECC9000106.md2.ford.com\qlik_prod8422\Descri...
4,FORDNA1\BALASRI,\\ECC9000106.md2.ford.com\qlik_prod8422\Descri...


## Add nodes to the graph

In [121]:
A = list(df["FIMGroup"].unique())
B = list(df["FileName"].unique())
node_list = list(set(A+B))
G = nx.Graph()
for i in node_list:
    G.add_node(i)

## Add edges to the graph 

In [122]:
for i,j in df.iterrows():
    G.add_edges_from([(j["FIMGroup"],j["FileName"])])

## Use spring_layout in networkx to position 

In [123]:
pos = nx.spring_layout(G, k=1, iterations=50)
for n, p in pos.items():
    G.node[n]['pos'] = p

In [124]:
edge_trace = go.Scatter(
    x=[],
    y=[],
    line=dict(width=0.5,color='#888'),
    hoverinfo='none',
    mode='lines')
for edge in G.edges():
    x0, y0 = G.node[edge[0]]['pos']
    x1, y1 = G.node[edge[1]]['pos']
    edge_trace['x'] += tuple([x0, x1, None])
    edge_trace['y'] += tuple([y0, y1, None])
node_trace = go.Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='RdBu',
        reversescale=True,
        color=[],
        size=15,
        colorbar=dict(
            thickness=10,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=0)))
for node in G.nodes():
    x, y = G.node[node]['pos']
    node_trace['x'] += tuple([x])
    node_trace['y'] += tuple([y])

In [125]:
for node, adjacencies in enumerate(G.adjacency()):
    node_trace['marker']['color']+=tuple([len(adjacencies[1])])
    node_info = adjacencies[0] +' # of connections: '+str(len(adjacencies[1]))
    node_trace['text']+=tuple([node_info])

In [126]:
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='network connections',
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="No. of connections",
                    showarrow=False,
                    xref="paper", yref="paper") ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))
fig.show()